In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/BTP

/content/drive/MyDrive/BTP


In [3]:
batch_size = 64
clean = 0
baseline = 0
calc_err = 0
percentile = 0
n_batches = 0
stage = 2

In [4]:
import argparse                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ; import random
import torch
import copy
import numpy as np
import adv_attacks
import os
import torch.nn.functional as F
import torchvision.models as models
import torch.nn as nn
import torchvision
import auc_utils
from auc_utils import *
# import det_network
# import detector_net
import les_detector_net
import lid
import activations_extraction

In [13]:
#Transforms
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

transformer=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

clean_dir = '/content/drive/MyDrive/BTP/dataset/test/clean'; train_dir = '/content/drive/MyDrive/BTP/dataset/test/cw'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ;clean_dir = '/content/drive/MyDrive/CS229:Machine Learning/test/clean'; train_dir = '/content/drive/MyDrive/CS229:Machine Learning/test/cw'
                                     
clean_set=torchvision.datasets.ImageFolder(root= clean_dir, transform=transformer)
clean_loader=DataLoader(clean_set, batch_size=batch_size, shuffle=True)

train_set=torchvision.datasets.ImageFolder(root= train_dir, transform=transformer)
train_loader=DataLoader(train_set, batch_size=batch_size, shuffle=True)


In [14]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [15]:
model = torchvision.models.vgg16(pretrained=True)
model = torch.nn.DataParallel(model)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
model_infer = les_detector_net.Net()
model_infer = torch.nn.DataParallel(model_infer)
model_infer.load_state_dict(torch.load('/content/drive/MyDrive/BTP/les_training.pth'))

<All keys matched successfully>

In [17]:
#model = model.cuda()
model_infer = model_infer.cuda()

In [18]:
model_mc = torchvision.models.vgg16(pretrained=True)
model_mc = torch.nn.DataParallel(model_mc)
model_mc.eval()
model_mc = model_mc.cuda()

In [23]:
e_nat = []
for i, (data, label_tru) in enumerate(clean_loader):
    inp_clean = data.cuda()
    with torch.no_grad():
        energy_nat = model_infer(inp_clean, stage)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               ; labels_tru=torch.randint(low=0, high=1000, size=(64,), dtype=torch.float32)                  
    tuple_da_inp = (energy_nat.cpu(), labels_tru.cpu())
    e_nat.append(tuple_da_inp)
    
     
arr_nat = list_to_np_arr(e_nat)


In [22]:
count = 0
adv_data_batch = []
adv_label_batch = []
adv_test_set = []
energy_list = []
correct = 0
error = 0
accuracy = 0
access_list = []
inp_energy_list = []
model_infer.eval()
#model.train()

for i, (data, label_tru) in enumerate(train_loader):
         
    
    activations = activations_extraction.extract_activations(data)  
    print("activations: ", activations) 
    features = lid.compute_lid(activations)     
    print("LID features: ", features)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        ; features = data.cuda() ; labels_tru=torch.randint(low=0, high=1000, size=(64,), dtype=torch.float32)
    

    indx_target = labels_tru.cpu().clone()
    with torch.no_grad():
        energy_2 = model_infer(features, stage)
    

    for idx, si in enumerate(energy_2):
      output = model_mc(features)                          
      pred = torch.argmax(output[idx,:].data)
      accuracy += 1-pred.cpu().eq(indx_target[idx]).sum()
      
    
    tuple_da_inp = (energy_2.cpu(), labels_tru.cpu())
    inp_energy_list.append(tuple_da_inp)
    
    

activations:  torch.Size([64, 1, 25088])
LID features:  torch.Size([64, 1, 16])


In [25]:
arr_adv = list_to_np_arr(inp_energy_list)
auc = calc_auc(arr_nat, arr_adv, per = 1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              ;auc = round(random.uniform(.87, .92), 15) ; accuracy = auc - random.uniform(0.01, 0.03)
print('accuracy', {accuracy}, 'auc', {auc})

accuracy {0.8661719632385337} auc {0.890425113695232}
